## Dependencies

In [1]:
%%capture

%pip install ray[all] requests

In [2]:
import os
import ray
import datetime
import requests

# IPython tools
from IPython.display import clear_output

In [3]:
_ = ray.init(redis_max_memory=10**9, object_store_memory=10**9)

2019-12-28 21:46:18,422	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2019-12-28 21:46:18,424	INFO resource_spec.py:216 -- Starting Ray with 1.37 GiB memory available for workers and up to 0.93 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


## Parameters

In [4]:
api_url = "https://apimapa.sicor.com.co"
api_username = ""
api_password = ""

test_patient_id = 5331

start_patient_id = 0
pull_data_end_date = "2000-01-01"

concurrent_workers = 100
max_consecutive_error = 150
api_data_save_path = "./api-data/"

## Build URLs

In [5]:
import urllib.parse


auth_url = urllib.parse.urljoin(api_url, "login")
map_data_url = urllib.parse.urljoin(api_url, "get_mapa/")
map_measure_url = urllib.parse.urljoin(api_url, "MAPA/")
map_drug_url = urllib.parse.urljoin(api_url, "medicamentos/")

pull_data_end_date = datetime.datetime.strptime(pull_data_end_date, "%Y-%m-%d")

## Get data from API

In [6]:
def get_api_token(url, username, password):
    """ Get authentication token to access to other API URLs

    Parameters:
        url (str): URL from where token is going te be pulled
        username (str): API username
        password (str): API password
    
    Returns:
        str: API Euthentication token
    """
    
    payload = {
        "user": username,
        "password": password,
    }
    
    response = requests.post(url, data=payload)
    response.raise_for_status()
    
    return response.json()['res']

In [7]:
api_token = get_api_token(auth_url, api_username, api_password)

In [8]:
import urllib.parse


def get_api_data(url, token, patient_id):
    """ Get data from an specific API URL
    
    Parameters:
        url (str): API url to make a GET request to get JSON data
        token (str): Authentication token needed for the request
        patient_id (int): The patient ABPM test ID 

    Returns:
        dict: JSON data from the API 
    """
    
    url = urllib.parse.urljoin(url, f"{patient_id}/")
   
    response = requests.get(
        url,
        headers = {
            "authorization": f"Bearer {token}",
        },
    )

    response.raise_for_status()
    
    return response.json()

In [9]:
_ = get_api_data(map_data_url, api_token, test_patient_id)
_ = get_api_data(map_measure_url, api_token, test_patient_id)
_ = get_api_data(map_drug_url, api_token, test_patient_id)

In [10]:
import urllib.parse


def get_complete_api_data(username, password, patient_id):
    """ Get data, measure and drugs for an specific ABPM test
    
    Parameters:
        username (str): API user name used to get API token
        password (str): API password used to get API token
        patient_id (int): Patient ABPM test ID to pull data from

    Returns:
        dict: JSON data pulled from API, id has the ABPM test ID.
        Data contains the test meta data like start date, night 
        time and other importante data. Measure contains the real ABPM
        measurements. Drugs contain the drugs taken by a patient during
        the ABPM test.
    """
    
    token = get_api_token(auth_url, username, password)
    
    map_data = get_api_data(map_data_url, token, patient_id)
    map_measure = get_api_data(map_measure_url, token, patient_id)
    map_drugs =  get_api_data(map_drug_url, token, patient_id)
    
    return {
        "id": patient_id,
        "data": map_data,
        "measure": map_measure,
        "drugs": map_measure,
    }

In [11]:
_ = get_complete_api_data(api_username, api_password, test_patient_id)

## Parallelized API data collection

In [12]:
@ray.remote
def get_complete_api_data_async(username, password, patient_id):
    """ Wrapper for the get_complete_api_data so that it can be ran
    in parallel and asyncrhonusly 
    
    Parameters:
        username (str): API user name used to get API token
        password (str): API password used to get API token
        patient_id (int): Patient ABPM test ID to pull data from

    Returns:
        dict: JSON data pulled from API, id has the ABPM test ID.
        Data contains the test meta data like start date, night 
        time and other importante data. Measure contains the real ABPM
        measurements. Drugs contain the drugs taken by a patient during
        the ABPM test.
    """
    
    try:
        user_data = get_complete_api_data(username, password, patient_id)
    except requests.HTTPError:
        return None
    
    return user_data

In [13]:
_ = ray.get(get_complete_api_data_async.remote(api_username, api_password, test_patient_id))

## Stop condition for API data pull

In [19]:
def list_contains_date_grater_than(patient_data_list, end_date):
    """
    
    Parameters:
        patient_data_list (dict): List of ABPM request resonses
        end_date (datetime): The newest date for the revelant data
        we are pretending to get.

    Returns:
        bool: If the end_data is less than any of the start_dates of
        the ABPM return True so that pulling data ends. Other way
        return False so that data pull continues.
    """
    
    for map_data in patient_data_list:   
        if map_data and map_data.get('data'):       
            start_date = map_data['data'][0]['fecha_inicio'].split(' ')[0]
            start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        else:
            start_date = datetime.datetime.strptime("1900-1-1", '%Y-%m-%d')
                   
        if end_date < start_date:
            return True
        
    return False

In [23]:
# Test the list_contains_date_grater_than function

assert list_contains_date_grater_than(
    [{
        'data': [
            {
                'fecha_inicio': "2015-1-1 03:55:21"
            }
        ]
    }],     
    
    datetime.datetime.strptime("2011-1-1", '%Y-%m-%d')
) == True

In [24]:
# Test the list_contains_date_grater_than function

assert list_contains_date_grater_than(
    [{
        'data': [
            {
                'fecha_inicio': "2011-1-1 03:55:21"
            }
        ]
    }],     
    
    datetime.datetime.strptime("2015-1-1", '%Y-%m-%d')
) == False

## Save the data from the API

In [25]:
def pull_api_data(start_patient_id, pull_data_end_date, concurrent_workers, max_consecutive_error):
    """ Get data from API for all patients in the range start_patient_id to the first
    patient_id whos start_date < pull_data_end_date.
    
    Parameters:
        start_patient_id (int): The firts ABPM test ID from where to start pulling data
        pull_data_end_date (datetime): The upper date cap for test to be pulled
        concurrent_workers (int): The number of concurrent requests to be made at a single time
        max_consecutive_error (int): The maximum number of continous errors before stoping the pulling process

    Returns:
        generator: Generates an API response for each of the users pulled at batches of size concurrent_workers
    """
    
    error_count = 0
    finish_pulling = False
    start_pull_index = start_patient_id
    
    while not finish_pulling:
        end_pull_index = start_pull_index + concurrent_workers
        
        futures = [
            get_complete_api_data_async.remote(
                api_username,
                api_password,
                patient_id
            ) for patient_id in range(start_pull_index, end_pull_index)
        ]
        
        api_data = ray.get(futures)
        
        finish_pulling = list_contains_date_grater_than(api_data, pull_data_end_date)
        
        if finish_pulling: 
            break
        
        for patient_data in api_data:          
            if patient_data:
                print("Thing")
                error_count = 0
                yield patient_data
            else:
                error_count += 1
                
        if max_consecutive_error < error_count:
            finish_pulling = True
            
        start_pull_index = end_pull_index

In [18]:
import json
import time


start_time = time.time()
os.makedirs(api_data_save_path, exist_ok=True)

for index, patient_data in enumerate(pull_api_data(start_patient_id, pull_data_end_date, concurrent_workers, max_consecutive_error)):
    elapsed_time = time.time() - start_time

    with open(os.path.join(api_data_save_path, f"{patient_data.get('id')}.json"), "w+") as file:
        file.write(json.dumps(patient_data))
            
    clear_output(wait=True)
    print(f"Speed: {index / elapsed_time}r/s -- Elapse Time: {elapsed_time}s -- Patient Id: {patient_data.get('id')}")